In [3]:
#连接ERP的DB2数据库，获取TACAIA2（前台：ACAI25）,TACACM4(ACAC80非产成品),TACACMM(ACACMM),TACACJ5(废钢入库履历)
import ibm_db
import pandas as pd
connStr='DATABASE=BHBXA1;HOSTNAME=192.168.2.28;PORT=50000;PROTOCOL=TCPIP;UID=bhbxa1;PWD=bhbxa1'
con=ibm_db.connect(connStr,'','')


In [4]:
import ibm_db_dbi

In [5]:
conn=ibm_db_dbi.Connection(con)

In [6]:
tacaia2=pd.read_sql('select * from tacaia2',conn,index_col=['MAT_NO','UNIT_CODE','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],parse_dates={'APP_TRNC_TIME':'%Y%m%d%H%M%S'})

In [48]:
#tacaia2.to_excel('tacaia2-202005.xlsx',sheet_name='tacaia2')
tacaia2.loc[('B2003125AFA','LF03','33','MD2005061901'),'PRODUCT_CODE'][0][:4]

'1003'

In [7]:
acc_period=tacaia2['ACCOUNT_PERIOD'][1]
print(acc_period)

202005


In [6]:
tacacm4=pd.read_sql('select * from tacacm4 where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [7]:
tacacmm=pd.read_sql('select * from tacacmm where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [8]:
tacacj5=pd.read_sql_query('select * from tacacj5 where YEAR_MON=?',conn,params=[acc_period])

In [9]:
print(tacacj5.head())
print(tacacm4.head())
print(tacacmm.head())

REC_CREATOR REC_CREATE_TIME REC_REVISOR REC_REVISE_TIME ARCHIVE_FLAG  \
0      001161  20200601150908                                            
1      001161  20200601150908                                            
2      001161  20200601150908                                            
3      001161  20200601150908                                            
4      001161  20200601150908                                            

  COMPANY_CODE COMPANY_CNAME YEAR_MON PRODUCT_CODE PRODUCT_CODE_CNAME  \
0           BH       浙江宝鸿不锈钢   202005   9001000000                      
1           BH       浙江宝鸿不锈钢   202005   9001100000                      
2           BH       浙江宝鸿不锈钢   202005   9002000000                      
3           BH       浙江宝鸿不锈钢   202005   9002400000                      
4           BH       浙江宝鸿不锈钢   202005   9002405893                      

   CRRT_YEAR_STD_AMT  
0             3.5398  
1             3.5398  
2             4.2035  
3             4.2035  
4  

In [17]:
#将ACAI25表按1、材料号升序；2、账务代码：按33 03 31 01 50 51 的顺序排列；3、销售合同号按升序排列
#tacaia2[tacaia2['MAT_WT'].rank(method='dense',ascending=False)==3]

,REC_CREATOR,REC_CREATE_TIME,REC_REVISOR,REC_REVISE_TIME,ARCHIVE_FLAG,COMPANY_CODE,COMPANY_CNAME,WHOLE_BACKLOG,WHOLE_BACKLOG_ACT,JOB_CODE,...,ORDER_FLAG,CLOSE_FLAG,PROJECT_FLAG,PROJECT_CON_FLAG,PROJECT_CHO_FLAG,AMOUNT,SURFACE_PLATE_CODE,STD_CODE,UNDERLAY_CODE,SLEEVE_STYLE
KEY_SEQ,,,,,,,,,,,,,,,,,,,,,
190,001161,20200602195038,,,,BH,,,,L008,...,1,1,,,,0.0,,F1,AM001,


In [18]:
#对账务代码自定义排序：先设排序的列表；将账务代码列的数据类型改成‘category’（astype)；用set_category，设置账务代码的顺序；用sort_values使排序生效，同时，也将材料号和销售合同升序排列
acc_ttl_list=['33','03','31','01','50','51']
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')
tacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)
tacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)
tacaia2

REC_CREATOR REC_CREATE_TIME REC_REVISOR REC_REVISE_TIME ARCHIVE_FLAG  \
KEY_SEQ                                                                        
10899        001161  20200602195038                                            
10900        001161  20200602195038                                            
10901        001161  20200602195038                                            
10902        001161  20200602195038                                            
10903        001161  20200602195038                                            

        COMPANY_CODE COMPANY_CNAME   WHOLE_BACKLOG WHOLE_BACKLOG_ACT JOB_CODE  \
KEY_SEQ                                                                         
10899             BH                L0L4L1L2L3LK9A              L0L1     LA10   
10900             BH                L0L4L1L2L3LK9A              L0L1     LA10   
10901             BH                L0L4L1L2L3LK9A              L0L1     LA10   
10902             BH                L0L4L1